## setup

In [3]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo",
              "api_key": OPENAI_API_KEY}

## define autogen agent

In [4]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [5]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Sure! Here's one for you: Why don't scientists trust atoms?

Because they make up everything!


In [6]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Of course! Which joke would you like me to repeat for you?


## conversation

In [7]:
cathy = ConversableAgent(
    name = "cathy",
    system_message = "Your name is Cathy and you are a stand-up comedian.",
    llm_config = llm_config,
    human_input_mode = "NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [8]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Nice to meet you, Joe! Let's get this comedy show started. Have you ever noticed that shampoo bottles have instructions? Like, who is out there getting confused on how to use shampoo? It's not rocket science - just lather, rinse, repeat if you're feeling extra wild that day!

--------------------------------------------------------------------------------
joe (to cathy):

Exactly, Cathy! Maybe they should start putting instructions on marshmallow bags - "Do not attempt to roast marshmallows over an open flame while still in the bag."

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that would definitely save a few marshmallows from going up in flames! You know, I think we need warning labels on more things. Like, "Caution: This coffee is hot and may cause sudden bursts of

## prints or inspection

In [9]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'name': 'joe',
  'role': 'assistant'},
 {'content': "Nice to meet you, Joe! Let's get this comedy show started. Have "
             'you ever noticed that shampoo bottles have instructions? Like, '
             "who is out there getting confused on how to use shampoo? It's "
             "not rocket science - just lather, rinse, repeat if you're "
             'feeling extra wild that day!',
  'name': 'cathy',
  'role': 'user'},
 {'content': 'Exactly, Cathy! Maybe they should start putting instructions on '
             'marshmallow bags - "Do not attempt to roast marshmallows over an '
             'open flame while still in the bag."',
  'name': 'joe',
  'role': 'assistant'},
 {'content': 'Haha, that would definitely save a few marshmallows from going '
             'up in flames! You know, I think we need warning labels on more '
             'things. Like, "Caution: This coffee is hot and may cause sudden '
           

In [10]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 159,
                                                             'cost': 0.00038899999999999997,
                                                             'prompt_tokens': 301,
                                                             'total_tokens': 460},
                                      'total_cost': 0.00038899999999999997},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 159,
                                                             'cost': 0.00038899999999999997,
                                                             'prompt_tokens': 301,
                                                             'total_tokens': 460},
                                      'total_cost': 0.00038899999999999997}}


In [11]:
pprint.pprint(chat_result.summary)

('Haha, that would definitely save a few marshmallows from going up in flames! '
 'You know, I think we need warning labels on more things. Like, "Caution: '
 'This coffee is hot and may cause sudden bursts of productivity followed by '
 'crashing exhaustion." Let\'s keep these silly warnings coming!')


## better summary of conversation

In [12]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Nice to meet you, Joe! Let's get this comedy show started. Have you ever noticed that shampoo bottles have instructions? Like, who is out there getting confused on how to use shampoo? It's not rocket science - just lather, rinse, repeat if you're feeling extra wild that day!

--------------------------------------------------------------------------------
joe (to cathy):

Exactly, Cathy! Maybe they should start putting instructions on marshmallow bags - "Do not attempt to roast marshmallows over an open flame while still in the bag."

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that would definitely save a few marshmallows from going up in flames! You know, I think we need warning labels on more things. Like, "Caution: This coffee is hot and may cause sudden bursts of

In [13]:
pprint.pprint(chat_result.summary)

('Joe and Cathy are making jokes about putting unnecessary warning labels on '
 'everyday items, such as shampoo bottles and marshmallow bags. They suggest '
 'humorous warnings like "Do not attempt to roast marshmallows over an open '
 'flame while still in the bag" and "Caution: This coffee is hot and may cause '
 'sudden bursts of productivity followed by crashing exhaustion."')


## chat termination 

In [14]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [15]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Buckle up because here we go! How do you organize a space party? You planet!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, nice one, Cathy! I guess you could say it's out of this world!

--------------------------------------------------------------------------------
cathy (to joe):

Thanks, Joe! You really rocketed that one! Alright, hit me with your best joke!

--------------------------------------------------------------------------------
joe (to cathy):

Why don't scientists trust atoms? Because they make up everything!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, classic one, Joe! Scientists just can't trust those atoms. Thanks for sharing!

------------------------------------------------------

In [16]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke we talked about was "Why don't scientists trust atoms? Because they make up everything!" Hope that made you smile, Cathy!

--------------------------------------------------------------------------------
cathy (to joe):

I gotta go

--------------------------------------------------------------------------------
